<a href="https://colab.research.google.com/github/timlac/emotional-recognition/blob/main/video_A220_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import os
import h5py
from multiprocessing import cpu_count
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
class TimeSeriesDataset(Dataset):

    def __init__(self):
        path = "/content/drive/My Drive/datasets/openface/video_data_low_level_A220.hdf5"

        f = h5py.File(name=path, mode='r')
        self.x = f['x']
        self.y = f['y']
        self.n_samples = self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [4]:
def train_test_split(dataset):
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train, test = random_split(dataset, [train_size, test_size])
    return train, test

In [5]:
dataset = TimeSeriesDataset()
train_set, test_set = train_test_split(dataset)

train_loader = DataLoader(dataset=train_set, batch_size=20, shuffle=True, num_workers=cpu_count())
test_loader = DataLoader(dataset=test_set, batch_size=20, shuffle=True, num_workers=cpu_count())

In [6]:
x_0, y_0 = iter(train_loader).next()
print("batch shape x ", x_0.shape)
print("batch shape y: ", y_0.shape)
print("number of batches train: ", len(train_loader))
print("number of batches test: ", len(test_loader))

batch shape x  torch.Size([20, 1084, 17])
batch shape y:  torch.Size([20])
number of batches train:  15
number of batches test:  4


## RNN

In [7]:
class RNN(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # x = batch_size, seq, input_size
        self.fc = nn.Linear(hidden_size, num_classes)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.rnn(x, h0)

        out = out[:, -1, :]

        out = self.fc(out)

        return out

In [8]:
hidden_size = 128
num_classes = 44
num_epochs = 100
learning_rate = 0.001

input_size = 17
sequence_length = 1084
num_layers = 2

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

model

RNN(
  (rnn): RNN(17, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=44, bias=True)
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (batch_x, labels) in enumerate(train_loader):
        batch_x = batch_x.to(device)

        labels = labels.to(device)

        outputs = model(batch_x)

        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            with torch.no_grad():
                n_correct = 0
                n_samples = 0

                for batch_x, labels in test_loader:
                    batch_x = batch_x.to(device)
                    labels = labels.to(device)

                    outputs = model(batch_x)
                    _, predicted = torch.max(outputs.data, 1)
                    n_samples += labels.size(0)
                    n_correct += (predicted == labels).sum().item()

                acc = n_correct / n_samples
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f} Accuracy: {acc}')

Epoch [1/100], Step [5/15], Loss: 3.7698 Accuracy: 0.0
Epoch [1/100], Step [10/15], Loss: 3.8097 Accuracy: 0.0
Epoch [1/100], Step [15/15], Loss: 3.8003 Accuracy: 0.0
Epoch [2/100], Step [5/15], Loss: 3.7913 Accuracy: 0.014084507042253521
Epoch [2/100], Step [10/15], Loss: 3.8220 Accuracy: 0.014084507042253521
Epoch [2/100], Step [15/15], Loss: 3.8717 Accuracy: 0.014084507042253521
Epoch [3/100], Step [5/15], Loss: 3.8000 Accuracy: 0.014084507042253521
Epoch [3/100], Step [10/15], Loss: 3.8016 Accuracy: 0.04225352112676056
Epoch [3/100], Step [15/15], Loss: 3.8792 Accuracy: 0.04225352112676056
Epoch [4/100], Step [5/15], Loss: 3.7790 Accuracy: 0.04225352112676056
Epoch [4/100], Step [10/15], Loss: 3.8026 Accuracy: 0.04225352112676056
Epoch [4/100], Step [15/15], Loss: 3.9136 Accuracy: 0.028169014084507043
Epoch [5/100], Step [5/15], Loss: 3.8005 Accuracy: 0.028169014084507043
Epoch [5/100], Step [10/15], Loss: 3.7967 Accuracy: 0.028169014084507043
Epoch [5/100], Step [15/15], Loss: 3.9

# LSTM

In [12]:
class LSTMClassifier(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def create_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.to(device) for t in (h0, c0)]

    def forward(self, x):
        h0, c0 = self.create_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))

        out = self.relu(out)
        
        # take only the last timestep of the output from the LSTM
        out = self.fc(out[:, -1, :])
        
        out = self.sigmoid(out)

        return out

In [15]:
hidden_size = 128
num_classes = 44
num_epochs = 100
learning_rate = 0.001

input_size = 17
sequence_length = 1084
num_layers = 2


model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)
model = model.to(device)
model

LSTMClassifier(
  (rnn): LSTM(17, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=44, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)

In [16]:
criterion = nn.CrossEntropyLoss()
opt = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (batch_x, labels) in enumerate(train_loader):
        batch_x = batch_x.to(device)

        labels = labels.to(device)

        outputs = model(batch_x)

        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            with torch.no_grad():
                n_correct = 0
                n_samples = 0

                for batch_x, labels in test_loader:
                    batch_x = batch_x.to(device)
                    labels = labels.to(device)

                    outputs = model(batch_x)
                    _, predicted = torch.max(outputs.data, 1)
                    n_samples += labels.size(0)
                    n_correct += (predicted == labels).sum().item()

                acc = n_correct / n_samples
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f} Accuracy: {acc}')

Epoch [1/100], Step [5/15], Loss: 3.7879 Accuracy: 0.014084507042253521
Epoch [1/100], Step [10/15], Loss: 3.7873 Accuracy: 0.014084507042253521
Epoch [1/100], Step [15/15], Loss: 3.7657 Accuracy: 0.014084507042253521
Epoch [2/100], Step [5/15], Loss: 3.7887 Accuracy: 0.014084507042253521
Epoch [2/100], Step [10/15], Loss: 3.7830 Accuracy: 0.014084507042253521
Epoch [2/100], Step [15/15], Loss: 3.7803 Accuracy: 0.014084507042253521
Epoch [3/100], Step [5/15], Loss: 3.7833 Accuracy: 0.014084507042253521
Epoch [3/100], Step [10/15], Loss: 3.7862 Accuracy: 0.014084507042253521
Epoch [3/100], Step [15/15], Loss: 3.7596 Accuracy: 0.014084507042253521
Epoch [4/100], Step [5/15], Loss: 3.7842 Accuracy: 0.014084507042253521
Epoch [4/100], Step [10/15], Loss: 3.7822 Accuracy: 0.014084507042253521
Epoch [4/100], Step [15/15], Loss: 3.7820 Accuracy: 0.014084507042253521
Epoch [5/100], Step [5/15], Loss: 3.7869 Accuracy: 0.014084507042253521
Epoch [5/100], Step [10/15], Loss: 3.7906 Accuracy: 0.01